This notebook is used for the sole purpose of gathering some information during experiment 2 
The function calls the Mongodb where the information for the hyperopt algorithm is stored. With this code I can check if everything is working as I expected (and checking every 5 min if the accuracy is already improved:)) 

In [ ]:
experiment_date = "6_25"


In [ ]:
from run_experiments.experiment_2 import make_space
from hyperopt import space_eval
from LJT_database.firestore_codes import  add_update, retrieve_updates
import pandas as pd
from hyperopt.mongoexp import MongoTrials
import pymongo
import dns

client = pymongo.MongoClient("mongodb://34.66.233.70:5000/")
mydb = client.eval_db
collection = mydb.jobs

#collection.drop()
   
coin="ETHBTC"
algorithm = "LSTM"
settings_experiment_2 = retrieve_updates(dataset=f"{coin}_{algorithm}_experiments",
                                    document="experiment2_settings")

settings = retrieve_updates(dataset=f"{coin}_{algorithm}_experiments",
                                            document="experiment_general_settings")

space = make_space(settings_experiment_2)

all_data = []
for doc in collection.find():
    if doc["exp_key"] == experiment_date:
        results = doc['result']
        tests = space_eval(space, {key: value[0] for key, value in doc['misc']['vals'].items()})
        data = {**results, **tests}
        data["experiment_date"] = settings["experiment_date"]
        all_data.append(data)

In [ ]:
x = pd.DataFrame(all_data)

In [ ]:
x.head()

In [ ]:
print(len(x))
x.sort_values('accuracy_val').iloc[-50:-1]

In [ ]:
x.sort_values("loss")[["accuracy_val",'highest_train_ac','optimizer','dropout','activation_function','learning_rate','epochs','batch_size']].head(150)

In [ ]:
#plot the loss over time (opposite of accuracy)
x.iloc[:]['loss'].plot.line()